# Introduction
To do downstream analysis later in the project we need:
1. Geolocation of where the sample was taken
2. Source that the sample was isolated from
3. Date on which the sample was collected
4. Filter out lab strains

None of these things have a column in any of the tables from the NCBI database. See this [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6380228/) that describes how metadata in NCBI sucks.
 
However, there is a column called 'sample_attribute' in the SRA and Sample table where a submitter can add additional information about a sample. As 'sample_attribute' does not require a specific format or specific information. The information found there varies greatly between samples. Some organizations (rivm) that submit data to the NCBI have a consisted format for this column which then also varies per organization, others do not. This makes it very challenging to extrapolate the information mentioned above for all samples. In this notebook we attempt to extract this information.

In [ ]:
import pandas as pd
import pyarrow.feather as feather
from collections import defaultdict

Functions written for this notebook are stored in wrangling_funcs.py. Please look there for documentation and tests.

In [ ]:
import wrangling_funcs

# Reading in the data
---

R has a nice package called SRAdb that you can use to query the NCBI database. However, I prefer working in Python. So we are querying the data in R using SRAdb and then exporting it in feather format for use here. There might be a way to directly get a dump of the SRA database and query it without using SRAdb. I will look into this.

The default index of a dataframe is not useful to us. Instead, we use the run_accession, these should be unique. This way we can keep track when we split the metadata into a separate dataframe.

In [ ]:
file_path = '../../results/SRA.feather'
data = feather.read_feather(file_path)
# data = feather.read_feather(snakemake.input[0])

metadata_df = pd.DataFrame(data)
metadata_df = metadata_df.convert_dtypes()
metadata_df.set_index('run_accession', inplace=True)

print(f'---Number of rows: {metadata_df.shape[0]}, Number of columns: {metadata_df.shape[1]}---')
metadata_df.head()

# Finding metadata in the sample_attribute
---

All the metadata we are interested in is contained in the 'sample_attribute' column. From what we could see most of the information in this column is split by '||' characters. The information between these characters is then often split using ':'. We will use this to make key value pairs which we will then turn into a dataframe.

In [ ]:
sample_attribute = metadata_df['sample_attribute']
faulty_lines = []
correct_lines = []

for line, identity in zip(sample_attribute, sample_attribute.index):
    line = line.split("||")
    line_items = defaultdict(list)
    for subitem in line:
        try:
            key, value = subitem.split(': ', 1)
            strip_key = wrangling_funcs.clean_string(key)
            strip_value = value.strip()
            line_items[strip_key] = strip_value
            line_items['run_accession'] = identity
        except ValueError:
            faulty_lines.append((identity, line))
    correct_lines.append(line_items)

smpl_att_df = pd.DataFrame(correct_lines)
smpl_att_df = smpl_att_df.convert_dtypes()
smpl_att_df.set_index('run_accession', inplace=True)

print(f'---Number of rows: {smpl_att_df.shape[0]}, Number of columns: {smpl_att_df.shape[1]}---')
smpl_att_df.head()

In [ ]:
smpl_att_df

### Replacing missing value synonyms with NA
Many of the entries in the database contain words such as 'missing', 'not available', 'not applicable' instead of NaN values. Here we replace those values by NaN's. Key word selection was done based on what we saw when looking through the dataframe.

In [ ]:
na_synonyms = {r'^\*$', '^-$', r'^\.$', '^[Nn]one$', '^[Nn]an$', '^[Uu]nknown$', '(?i)^not[ _-]collected$', '(?i)^not[ _-]provided', r'^\?$', '^ $', '(?i)^not[ _-]applicable$', '^[Nn]a$',
         '^[Nn]o$', '^[Oo]ther$','^[Mm]is{1,3}ing$', '^[Uu]nspecified$', '^[Nn]ot[ ]available$', '^[Nn]ot[ :]available[:] not collected$', '^[Nn]ot[ :]available[:] to be reported later$'}

smpl_att_df_test = smpl_att_df.replace(to_replace=na_synonyms, value=pd.NA, regex=True)

print(f'---Number of rows: {smpl_att_df.shape[0]}, Number of columns: {smpl_att_df.shape[1]}---')
smpl_att_df.head()

### Searching for geographic data

There is no consistent column that contains the geolocation. To (hopefully) obtain the geolocation we use regex to find keywords in the column names of the dataframe. The matched columns are then combined in a single column while handling NaN values.

In [ ]:
geo_col_matches = wrangling_funcs.find_columns(['geo', 'geographic', 'country', 'continent'], smpl_att_df, ['longitude', 'latitude', 'depth'])
print(f'The following columns matched the keywords: {geo_col_matches}')
smpl_att_df = wrangling_funcs.combine_columns(smpl_att_df, list(geo_col_matches), 'inferred_location')
# meta_df.drop(geo_col_matches, inplace=True, axis=1)

smpl_att_df['inferred_continent'], smpl_att_df['inferred_country'], smpl_att_df['inferred_city'] = zip(*smpl_att_df['inferred_location'].map(wrangling_funcs.clean_geo))
smpl_att_df.drop('inferred_location', axis=1, inplace=True)

print(f'---Number of rows: {smpl_att_df.shape[0]}, Number of columns: {smpl_att_df.shape[1]}---')
smpl_att_df.head()

### Searching for the sample collection data

There is no consistent column that contains the date. To (hopefully) obtain the date we use regex to find keywords in the column names of the dataframe. The matched columns are then combined in a single column while handling NaN values.

In [ ]:
date_col_matches = wrangling_funcs.find_columns(['date', 'year'], smpl_att_df, ['update'])
print(f'The following columns matched the keywords: {date_col_matches}')
smpl_att_df = wrangling_funcs.combine_columns(smpl_att_df, list(date_col_matches), 'inferred_collection_year')
# meta_df.drop(date_col_matches, inplace=True, axis=1)


date = smpl_att_df['inferred_collection_year'].str.extract(r'^(\d{4})', expand=False) # Extract the year
smpl_att_df['inferred_collection_year'] = pd.to_numeric(date) # cast year to int
# meta_df['inferred_collection_year']

print(f'---Number of rows: {smpl_att_df.shape[0]}, Number of columns: {smpl_att_df.shape[1]}---')
smpl_att_df.head()

### Searching for sample isolation source
There is no consistent column that contains the isolation source. To (hopefully) obtain the isolation source we use regex to find keywords in the column names of the dataframe. The matched columns are then combined in a single column while handling NaN values.


In [ ]:
isolate_matches = wrangling_funcs.find_columns(['sample', 'source', 'environment', 'env', 'site'], smpl_att_df, ['name', 'provider', 'comment'])
print(isolate_matches)

smpl_att_df = wrangling_funcs.combine_columns(smpl_att_df, list(isolate_matches), "inferred_source")
# meta_df.drop(isolate_matches, inplace=True, axis=1)

smpl_att_df['inferred_source'] = smpl_att_df['inferred_source'].apply(wrangling_funcs.clean_source)

print(f'---Number of rows: {smpl_att_df.shape[0]}, Number of columns: {smpl_att_df.shape[1]}---')
smpl_att_df.head()

### First round removal of lab strains
We don't want sample from lab strains, so we filter out rows based on some keywords

In [ ]:
keywords_to_exclude = ["replicate", "mutant"]
smpl_att_df = smpl_att_df[~smpl_att_df.sample_comment.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]
smpl_att_df = smpl_att_df[~smpl_att_df.genotype.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]
smpl_att_df = smpl_att_df[~smpl_att_df.lab_experiment_type.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]

print(f'---Number of rows: {smpl_att_df.shape[0]}, Number of columns: {smpl_att_df.shape[1]}---')
smpl_att_df.head()

### Remove non relevant columns
We have a ton of columns and very few of them are actually usefull to us. Let's remove all not relevant columns

In [ ]:
smpl_att_df = smpl_att_df[['strain', 'inferred_collection_year', 'inferred_source', 'inferred_continent', 'inferred_country', 'inferred_city', 'geographic_location_latitude', 'geographic_location_longitude']]

print(f'---Number of rows: {smpl_att_df.shape[0]}, Number of columns: {smpl_att_df.shape[1]}---')
smpl_att_df.head()

# Combine sample_attribute metadata with rest of the data
---
Now that we have extracted the metadata that we wanted we can combine it back to the original dataframe. We only want to keep rows that have values for the collection_year/source/country because we require this downstream.

In [ ]:
combined_df = metadata_df.join(smpl_att_df)
cols = ['inferred_collection_year', 'inferred_source', 'inferred_country']
combined_df = combined_df.dropna(subset=cols)

print(f'---Number of rows: {combined_df.shape[0]}, Number of columns: {combined_df.shape[1]}---')
combined_df.head()

### Remove lab strain samples from combined dataframe
After combining the dataframes we have new columns that might give us more information if a particular run comes from a lab strain, so we filter again.


In [ ]:
keywords_to_exclude = ["replicate", "mutant"]

combined_df = combined_df[~combined_df.description.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]
combined_df = combined_df[~combined_df.study_title.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]
combined_df = combined_df[~combined_df.study_abstract.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]
combined_df = combined_df[~combined_df.study_description.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]

print(f'---Number of rows: {combined_df.shape[0]}, Number of columns: {combined_df.shape[1]}---')
combined_df.head()

### Remove samples submitted by the RIVM
Since we will already have the RIVM samples in the database there is no need to have them here.

In [ ]:
combined_df = combined_df[~combined_df.submission_center.str.contains("RIVM", na=False)] # Remove samples submitted by the RIVM

print(f'---Number of rows: {combined_df.shape[0]}, Number of columns: {combined_df.shape[1]}---')
combined_df.head()

### Replace missing value synonyms with NA
Once again after combining and obtaining new columns there are values in the rows that indicate NaNs

In [ ]:
na_synonyms = {r'^\*$', '^-$', r'^\.$', '^[Nn]one$', '^[Nn]an$', '^[Uu]nknown$', '(?i)^not[ _-]collected$', '(?i)^not[ _-]provided', r'^\?$', '^ $', '(?i)^not[ _-]applicable$', '^[Nn]a$',
               '^[Nn]o$', '^[Oo]ther$','^[Mm]is{1,3}ing$', '^[Uu]nspecified$', '^[Nn]ot[ ]available$', '^[Nn]ot[ :]available[:] not collected$', '^[Nn]ot[ :]available[:] to be reported later$'}
combined_df = combined_df.replace(to_replace=na_synonyms, value=pd.NA, regex=True)

print(f'---Number of rows: {combined_df.shape[0]}, Number of columns: {combined_df.shape[1]}---')
combined_df.head()

### Throw away empty columns
We want to filter out columns that only have NaN values so there is less cluter

In [16]:
combined_df = combined_df.dropna(axis=1, how='all')

print(f'---Number of rows: {combined_df.shape[0]}, Number of columns: {combined_df.shape[1]}---')
combined_df.head()

---Number of rows: 26104, Number of columns: 56---


,sra_ID,run_ID,run_alias,updated_date,spots,bases,run_center,experiment_name,run_attribute,experiment_ID,...,sradb_updated,scientific_name,strain,inferred_collection_year,inferred_source,inferred_continent,inferred_country,inferred_city,geographic_location_latitude,geographic_location_longitude
run_accession,,,,,,,,,,,,,,,,,,,,,
DRR065950,60940,60941,DRR065950,2023-06-18,163482,2261498784,<NA>,DRX060093,<NA>,55877,...,2023-12-09 17:25:46,Escherichia coli,MRY15-117,2012.0,"MIGS.ba,missing",Asia,Japan,NaN,<NA>,<NA>
DRR065949,60942,60940,DRR065949,2023-06-20,163482,1515099158,<NA>,DRX060092,<NA>,55875,...,2023-12-09 17:25:46,Escherichia coli,20Ec-P-124,2008.0,"MIGS.ba,missing",Asia,Japan,NaN,<NA>,<NA>
DRR065951,60944,60943,DRR065951,2023-06-21,163482,1398527654,<NA>,DRX060094,<NA>,55874,...,2023-12-09 17:25:46,Escherichia coli,MRY15-131,2013.0,"MIGS.ba,missing",Asia,Japan,NaN,<NA>,<NA>
DRR067778,62654,62654,DRR067778,2023-06-20,35065,247198598,<NA>,DRX061698,<NA>,57390,...,2023-12-09 17:25:46,Klebsiella pneumoniae,KP01,2014.0,"rectal swab,missing,not applicable,MIGS.ba.hum...",Asia,Thailand,Bangkok,<NA>,<NA>
DRR070681,65464,65468,DRR070681,2023-06-18,69700,383913853,<NA>,DRX064631,<NA>,60206,...,2023-12-09 17:25:46,Escherichia coli,M105,2015.0,"human blood,not applicable,MIGS.ba.human-assoc...",Asia,Myanmar,Yangon,<NA>,<NA>


In [17]:
combined_df

,sra_ID,run_ID,run_alias,updated_date,spots,bases,run_center,experiment_name,run_attribute,experiment_ID,...,sradb_updated,scientific_name,strain,inferred_collection_year,inferred_source,inferred_continent,inferred_country,inferred_city,geographic_location_latitude,geographic_location_longitude
run_accession,,,,,,,,,,,,,,,,,,,,,
DRR065950,60940,60941,DRR065950,2023-06-18,163482,2261498784,<NA>,DRX060093,<NA>,55877,...,2023-12-09 17:25:46,Escherichia coli,MRY15-117,2012.0,"MIGS.ba,missing",Asia,Japan,NaN,<NA>,<NA>
DRR065949,60942,60940,DRR065949,2023-06-20,163482,1515099158,<NA>,DRX060092,<NA>,55875,...,2023-12-09 17:25:46,Escherichia coli,20Ec-P-124,2008.0,"MIGS.ba,missing",Asia,Japan,NaN,<NA>,<NA>
DRR065951,60944,60943,DRR065951,2023-06-21,163482,1398527654,<NA>,DRX060094,<NA>,55874,...,2023-12-09 17:25:46,Escherichia coli,MRY15-131,2013.0,"MIGS.ba,missing",Asia,Japan,NaN,<NA>,<NA>
DRR067778,62654,62654,DRR067778,2023-06-20,35065,247198598,<NA>,DRX061698,<NA>,57390,...,2023-12-09 17:25:46,Klebsiella pneumoniae,KP01,2014.0,"rectal swab,missing,not applicable,MIGS.ba.hum...",Asia,Thailand,Bangkok,<NA>,<NA>
DRR070681,65464,65468,DRR070681,2023-06-18,69700,383913853,<NA>,DRX064631,<NA>,60206,...,2023-12-09 17:25:46,Escherichia coli,M105,2015.0,"human blood,not applicable,MIGS.ba.human-assoc...",Asia,Myanmar,Yangon,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERR9079443,7990656,7990127,SAL_LB9488AA_TR,2023-05-21,2301518,685852364,"Anthony Smith, NICD",<NA>,ENA-FIRST-PUBLIC: 2022-03-05 || ENA-LAST-UPDAT...,7622780,...,2023-12-09 17:25:46,Salmonella enterica,YA00197045,2020.0,"Stool,Human",Africa,South Africa,Western Cape,<NA>,<NA>
ERR9079436,7990669,7990120,SAL_LB9480AA_TR,2023-05-18,1776669,529447362,"Anthony Smith, NICD",<NA>,ENA-FIRST-PUBLIC: 2022-03-05 || ENA-LAST-UPDAT...,7622773,...,2023-12-09 17:25:46,Salmonella enterica,YA00230774,2020.0,"Blood culture,Human",Africa,South Africa,Eastern Cape,<NA>,<NA>
ERR9079429,7990698,7990113,SAL_LB9471AA_TR,2023-06-26,1991641,593509018,"Anthony Smith, NICD",<NA>,ENA-FIRST-PUBLIC: 2022-03-05 || ENA-LAST-UPDAT...,7622766,...,2023-12-09 17:25:46,Salmonella enterica,YA00230709,2020.0,"Stool,Human",Africa,South Africa,KwaZulu-Natal,<NA>,<NA>


## Write out files

In [0]:
combined_df.to_csv(snakemake.output[0], na_rep='NaN')